In [6]:
import os
import asyncio
import json
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI
import subprocess

In [7]:
# -----------------------------
# Tools Implementation
# -----------------------------
class ReadTool:
    def __init__(self, workspace_dir):
        self.workspace_dir = Path(workspace_dir)

    def run(self, filename):
        file_path = self.workspace_dir / filename
        if file_path.exists():
            return file_path.read_text()
        return f"File {filename} not found."

class WriteTool:
    def __init__(self, workspace_dir):
        self.workspace_dir = Path(workspace_dir)

    def run(self, filename, content):
        file_path = self.workspace_dir / filename
        file_path.write_text(content)
        return f"Written to {filename}"

class EditTool:
    def __init__(self, workspace_dir):
        self.workspace_dir = Path(workspace_dir)

    def run(self, filename, new_content):
        file_path = self.workspace_dir / filename
        if file_path.exists():
            file_path.write_text(new_content)
            return f"{filename} updated."
        return f"File {filename} does not exist."

class BashTool:
    def __init__(self, workspace_dir):
        self.workspace_dir = workspace_dir

    def run(self, command):
        try:
            result = subprocess.check_output(
                command,
                shell=True,
                text=True,
                cwd=self.workspace_dir,
                stderr=subprocess.STDOUT
            )
            return result
        except subprocess.CalledProcessError as e:
            return str(e)


class SessionNoteTool:
    def __init__(self, memory_file):
        self.memory_file = Path(memory_file)
        if not self.memory_file.exists():
            self.memory_file.write_text("[]")

    def record(self, category, content):
        notes = json.loads(self.memory_file.read_text())
        notes.append({"category": category, "content": content})
        self.memory_file.write_text(json.dumps(notes, indent=2))
        return "Note recorded."

class RecallNoteTool:
    def __init__(self, memory_file):
        self.memory_file = Path(memory_file)

    def recall(self, category=None):
        notes = json.loads(self.memory_file.read_text())
        if category:
            notes = [n for n in notes if n["category"] == category]
        return notes

In [8]:
# -----------------------------
# Agent Implementation
# -----------------------------
class Agent:
    def __init__(self, llm_client, system_prompt, tools, workspace_dir):
        self.llm_client = llm_client
        self.system_prompt = system_prompt
        self.tools = {tool.__class__.__name__: tool for tool in tools}
        self.user_messages = []
        self.workspace_dir = Path(workspace_dir)

    def add_user_message(self, message):
        self.user_messages.append(message)

    def run(self):
        user_prompt = "\n".join(self.user_messages)
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": user_prompt},
        ]

        max_steps = 15
        step = 0
        final_answer = None

        while step < max_steps:
            step += 1

            # 每轮开始前，把当前记忆（任务进度 + 错误）加入系统消息
            notes = self.tools['RecallNoteTool'].recall(category="progress") + self.tools['RecallNoteTool'].recall(category="error")
            progress_summary = "\n".join([f"[{n['category']}] {n['content']}" for n in notes])
            if progress_summary:
                # print(f"\n💾 Current session notes:\n{progress_summary}\n")
                messages.append({"role": "system", "content": f"Current task status:\n{progress_summary}"})


            # 调用 LLM
            response = self.llm_client.chat.completions.create(
                model="qwen3-coder-plus",
                messages=messages,
                stream=False,
                temperature=0
            )
            content = response.choices[0].message.content.strip()

            # print(f"content: {content}\n")
            # print("+"*40)

            # 尝试解析工具调用 JSON
            try:
                data = json.loads(content)
                tool_name = data.get("tool")
                args = data.get("args", {})

                if tool_name in self.tools:
                    tool = self.tools[tool_name]
                    print(f"\n🛠 Step {step}: Calling tool {tool_name} with args {args}")

                    # 执行工具
                    if hasattr(tool, "run"):
                        result = tool.run(**args)
                    elif hasattr(tool, "record"):
                        result = tool.record(**args)
                    elif hasattr(tool, "recall"):
                        result = tool.recall(**args)
                    else:
                        result = f"{tool_name} has no callable method."

                    # print(f"✅ Tool {tool_name} output:\n{result}")

                    # 自动记录写文件/Bash操作到进度或错误
                    note_tool = self.tools['SessionNoteTool']

                    if tool_name in ['WriteTool', 'EditTool']:
                        note_tool.record(category="progress", content=f"Tool {tool_name} executed with args {args}")

                    elif tool_name == 'BashTool':
                        # 如果 result 是异常对象或字符串包含 "returned non-zero" 就判定失败
                        if isinstance(result, Exception) or "returned non-zero" in str(result):
                            note_tool.record(category="error", content=f"BashTool failed with command '{args['command']}': {result}")
                            print(f"💾 Recorded BashTool error to memory: BashTool failed with command '{args['command']}': {result}")
                        else:
                            note_tool.record(category="progress", content=f"Tool {tool_name} executed with args {args}")

                    messages.append({"role": "assistant", "content": f"Tool {tool_name} output:\n{result}"})
                    continue
                else:
                    final_answer = content
                    break
            except json.JSONDecodeError:
                # JSON解析失败，当作最终回答
                final_answer = content
                break

        return final_answer

In [9]:
# -----------------------------
# Demo Full Agent
# -----------------------------
def demo_full_agent():
    print("\n" + "=" * 60)
    print("Full Mini Agent with Memory and Progress Logging")
    print("=" * 60)

    load_dotenv()
    api_key = os.getenv("DASHSCOPE_API_KEY")
    if not api_key:
        print("❌ DASHSCOPE_API_KEY not found in .env")
        return

    workspace_dir = "testworkdir"
    os.makedirs(workspace_dir, exist_ok=True)
    print(f"📁 Workspace: {workspace_dir}")

    # -----------------------------
    # System prompt with strict JSON format
    # -----------------------------
    system_prompt = """
You are a helpful AI assistant that performs tasks in a workspace using tools.

TOOLS:

1. ReadTool
   - JSON: {"tool": "ReadTool", "args": {"filename": "example.txt"}}

2. WriteTool
   - JSON: {"tool": "WriteTool", "args": {"filename": "example.txt", "content": "Hello world"}}

3. EditTool
   - JSON: {"tool": "EditTool", "args": {"filename": "example.txt", "new_content": "Updated content"}}

4. BashTool
   - JSON: {"tool": "BashTool", "args": {"command": "python calculator.py"}}
   - RULES:
     - Always run commands inside the workspace root directory.
     - Only call BashTool for steps that require executing scripts.
     - If the last memory entry is a BashTool error:
         1. Do NOT call any other tool (including SessionNoteTool).
         2. Do NOT output plain text.
         3. Analyze the error carefully and generate a corrected BashTool JSON call to retry.
         4. Repeat this process until the BashTool step succeeds.

5. SessionNoteTool
   - JSON: {"tool": "SessionNoteTool", "args": {"category": "project", "content": "This is a note"}}

6. RecallNoteTool
   - JSON: {"tool": "RecallNoteTool", "args": {"category": "project"}}

IMPORTANT INSTRUCTIONS:

- Always respond **only with valid JSON** when calling a tool.
- Before each action, check the last memory entry:
    - If it is a BashTool error, follow the rules above and retry BashTool.
    - Otherwise, continue with the next step of the task.
- When all steps are complete, respond with final answer in plain text.
- Example BashTool call:
  {"tool": "BashTool", "args": {"command": "python calculator.py"}}
- Example final answer (plain text):
  "All files created and the calculator script ran successfully."
"""


    client = OpenAI(
        api_key=api_key,
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
    )

    # Initialize tools
    tools = [
        ReadTool(workspace_dir=workspace_dir),
        WriteTool(workspace_dir=workspace_dir),
        EditTool(workspace_dir=workspace_dir),
        BashTool(workspace_dir=workspace_dir),
    ]
    memory_file = Path(workspace_dir) / "agent_memory.json"
    tools.extend([
        SessionNoteTool(memory_file=str(memory_file)),
        RecallNoteTool(memory_file=str(memory_file)),
    ])

    agent = Agent(
        llm_client=client,
        system_prompt=system_prompt,
        tools=tools,
        workspace_dir=workspace_dir
    )

    # -----------------------------
    # Task
    # -----------------------------
    task = """
    Please help me with the following tasks:

    1. Create a Python script called 'calculator.py' that:
        - Has functions for add, subtract, multiply, divide
        - Has a main() function that demonstrates usage
        - Includes proper docstrings and type hints

    2. Create a README.md file that:
        - Describes the calculator script
        - Shows how to run it
        - Lists the available functions

    3. Test the calculator by running it with bash

    4. Remember this project info:
        - Project: Simple Calculator
        - Language: Python
        - Purpose: Demonstration of agent capabilities
    """
    agent.add_user_message(task)
    result = agent.run()

    print("\n" + "=" * 60)
    print("✅ Agent completed!")
    print("=" * 60)
    # print(f"\nAgent's final response:\n{result}\n")

    # -----------------------------
    # Show workspace files
    # -----------------------------
    # workspace = Path(workspace_dir)
    # print("=" * 60)
    # print("📄 Created files in workspace:")
    # print("=" * 60)
    # for file in workspace.glob("*"):
    #     if file.is_file() and not file.name.startswith("."):
    #         print(f"\n📄 {file.name}:")
    #         print("-" * 60)
    #         content = file.read_text()
    #         lines = content.split("\n")[:20]
    #         print("\n".join(lines))
    #         if len(content.split("\n")) > 20:
    #             print("... (truncated)")
    #         print("-" * 60)

    # -----------------------------
    # Show memory
    # -----------------------------
    if memory_file.exists():
        notes = json.loads(memory_file.read_text())
        print(f"\n💾 Session notes recorded: {len(notes)}")
        for note in notes:
            print(f"  - [{note['category']}] {note['content'][:60]}...")

In [10]:
def main():
    demo_full_agent()

if __name__ == "__main__":
    main()


Full Mini Agent with Memory and Progress Logging
📁 Workspace: testworkdir

🛠 Step 1: Calling tool WriteTool with args {'filename': 'calculator.py', 'content': 'def add(a: float, b: float) -> float:\n    """Add two numbers together."""\n    return a + b\n\ndef subtract(a: float, b: float) -> float:\n    """Subtract second number from first number."""\n    return a - b\n\ndef multiply(a: float, b: float) -> float:\n    """Multiply two numbers together."""\n    return a * b\n\ndef divide(a: float, b: float) -> float:\n    """Divide first number by second number."""\n    if b == 0:\n        raise ValueError("Cannot divide by zero")\n    return a / b\n\ndef main():\n    """Demonstrate usage of calculator functions."""\n    print("Calculator Demo")\n    print(f"10 + 5 = {add(10, 5)}")\n    print(f"10 - 5 = {subtract(10, 5)}")\n    print(f"10 * 5 = {multiply(10, 5)}")\n    print(f"10 / 5 = {divide(10, 5)}")\n\nif __name__ == "__main__":\n    main()\n'}

🛠 Step 2: Calling tool WriteTool with 